135. 分发糖果

https://leetcode.cn/problems/candy/description/?envType=study-plan-v2&envId=top-interview-150

这是一道难题，但其传统方法并不那么难。它的 constraint 是，队列中的每个小孩，都满足如果这个小孩分数高，那么它的糖果理应更多。

这里的第一个难点在于将该约束显化为数学表示。一旦显化，就发现这实际上是两个约束，即（1）比左边小孩高（2）比右边小孩高；只要同时满足这两个约束，则总约束必然满足，这是分解法。自然，我们希望用两个遍历（从左到右、从右到左）解决这个问题。第二个难点在于，这两个过程是否独立的估计，很明显不是独立的。

不独立是可以证明的。首先，序列中只可能存在四种情况：
- [[升序], [升序]]
- [[升序], [降序]]
- [[降序], [升序]]
- [[降序], [降序]]

如果序列的某一部分存在先升后降 [[升序], [降序]]，则反向过程对应是 [[降序], [升序]] 的；那么这部分在左右遍历的两次过程中，都会被运算处理，因此会互相影响。而其他三种可能都不会，注意 [[降序], [升序]] 对于左右两个遍历过程分别处理了不同的部分。

因此这引出了本题的第一个坑，也是我第一次没写对的地方，就是 candies[i] = max(candies[i], candies[i + 1] + 1) 这里。因为左到右的遍历已经将降序之前的升序处理完毕，而**升序的最后一个值也是降序的第一个值，对应的就是从右到左过程升序的最后一个值**。这个值如果满足从右到左的升序，那么保留即可；如果不满足，说明还要更高，因此是 max 操作。至此也解释了为什么本题不能用滑动窗口，因为信息量不够，在没有从最右侧反向遍历时，我们永远不知道拐点在哪。第二个坑就是 range(1, n) 函数，写着写着就忘了这个范围是 [1, 2, ..., n - 1]。

In [ ]:
from typing import List

class Solution:
    def candy(self, ratings: List[int]) -> int:
        n = len(ratings)
        candies = [1] * n

        # left -> right
        for i in range(1, n):
            if ratings[i] > ratings[i - 1]:
                candies[i] = candies[i - 1] + 1

        # right -> left
        for i in range(n - 2, -1, -1):
            if ratings[i] > ratings[i + 1]:
                candies[i] = max(candies[i], candies[i + 1] + 1)

        return sum(candies)

显而易见时间复杂度是 o(n)，空间复杂度是 o(n)。本题说是还有个 o(1) 的解法。

在开始之前，请你想一想，对于 [[升序], [降序]] 的峰顶，我们已经知道了应该怎样分配，即 max 两个上升趋势的值；那么对于 [[降序], [升序]] 的谷底呢？答案其实非常简单，就是分配 1 呀！因为它比左右两侧的 rating 都小，自然只分配一颗糖果即可满足需求。

这道题的最后这种做法也叫坡度计数法。几个关键点
- 状态机思想，即上坡 s1、平路 s2、下坡 s3 三种状态，彼此之间可以互相转移；
- 关键观察，糖果分配只和“上坡连续长度”和“下坡连续长度”有关，这是本题的关键，也就是说一旦有平坦处，都分配 1 即可，这个要想明白；对于谷点也是分配 1 即可；
- 峰顶的情况比较复杂，也是本题最难想的点，如果上坡长度小于下坡长度，就需要减掉之前上坡算出来的 peak，否则就减掉下坡算出来的 peak；这里 if last_up >= down 代码无论如何都可以 cover。

In [ ]:
class Solution:
    def candy(self, ratings: List[int]) -> int:
        n = len(ratings)

        if n <= 1:
            return n

        total = 1
        up = 0
        down = 0
        last_up = 0
        
        for i in range(1, n): # state machine
            if ratings[i] > ratings[i - 1]: # state 1
                up += 1
                last_up = up
                down = 0
                total += (up + 1)
            elif ratings[i] == ratings[i - 1]: # state 2
                up, down, last_up = 0, 0, 0
                total += 1
            else:  # state 3
                down += 1
                if last_up >= down:
                    total -= 1
                up = 0
                total += (down + 1)

        return total

这里作为一个小的变体，如果倒序来做也是一样的（对称的）。

In [ ]:
class Solution:
    def candy(self, ratings: List[int]) -> int:
        n = len(ratings)

        if n <= 1:
            return n

        total = 1
        up = 0
        down = 0
        last_up = 0
        
        for i in range(n - 2, -1, -1):
            if ratings[i] > ratings[i + 1]:
                up += 1
                last_up = up
                down = 0
                total += (up + 1)
            elif ratings[i] == ratings[i + 1]:
                up, down, last_up = 0, 0, 0
                total += 1
            else:
                down += 1
                if last_up >= down:
                    total -= 1
                up = 0
                total += (down + 1)

        return total

总结一下，分糖果这道题的本质是 local 约束问题。经过分析转化为左向右、右向左两种情况，然后考虑左右在顶点处的冲突并解决。一步一步将信息量转化为一行行代码。

再进一步观察，得到新的信息量（1）分配糖果数量的计算只和上下坡有关（2）谷底和平路都分配 1（3）顶峰处两次计算取坡度更长的那个值，因此要减去坡度短的过程算出来的 peak。如此用这些信息构造代码，自然而然用 state machine 最方便。